In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import keras
from keras.datasets import mnist
import numpy as np
import cv2


Using TensorFlow backend.


### 0 使用GPU进行训练

In [1]:
import tensorflow as tf
# print('tensorflow version:',tf.__version__)  # 查看TensorFlow的版本
# print('===================================================')
# print('cuda available:',tf.test.is_built_with_cuda()) # 判断CUDA是否可用
# print('===================================================')
# print(tf.test.is_gpu_available())  # 查看cuda、TensorFlow_GPU和cudnn(选择下载，cuda对深度学习的补充)版本是否对应
# print('===================================================')
gpus = tf.config.experimental.list_physical_devices(device_type='GPU') # 查看可用GPU
# print(gpus)
import os
#选择使用某一块或多块GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # =右边"0,1",代表使用标号为0,和1的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # =右边"0",代表使用标号为0的GPU
# 查看可用GPU的详细信息
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
#  这时候在运行相应的代码，就可以看到在GPU上运行了。可以通过任务管理器-性能处查看GPU使用率。

### 1 数据预处理

In [3]:
# 首先加载MNIST数据集。然后，通过循环遍历每个图像，使用OpenCV库的cvtColor函数将灰度图像转换为RGB格式（因为AlexNet需要RGB图像作为输入）。
# 接下来，使用resize函数将图像大小调整为227x227。调整后的图像被添加到相应的列表中。最后，使用np.array将图像列表转换为NumPy数组。
# 由于MNIST数据集中的图像是灰度图像（单通道），因此在转换为RGB格式之前需要将其从二维数组调整为三维数组（添加一个额外的维度）。
# 这可以通过使用np.expand_dims函数来实现。

# 加载手写数字数据集（MNIST）
# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
(train_images, train_labels), (test_images, test_labels) =mnist.load_data("mnist.npz") # 加载本地的mnist数据集

# 将图像大小调整为227x227
resized_train_images = []
resized_test_images = []

# 由于电脑性能有限，未训练全部mnist数据集，仅选择其中一部分为keras_alexnet做工程验证

train_labels=train_labels[:6000]
test_labels=test_labels[:1000]

for image in train_images[:6000]:
    # 将图像从灰度转换为RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) 
    # 调整图像大小为227x227
    resized_image = cv2.resize(image_rgb, (227, 227))
    # 添加调整后的图像到列表
    resized_train_images.append(resized_image)

for image in test_images[:1000]:
    image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    resized_image = cv2.resize(image_rgb, (227, 227))
    resized_test_images.append(resized_image)

# 将图像列表转换为 NumPy数组 并 归一化
resized_train_images = np.array(resized_train_images).astype('float32') / 255.0
resized_test_images = np.array(resized_test_images).astype('float32') / 255.0

# 输出调整后图像的形状
print("调整后的训练图像形状:", resized_train_images.shape)
print("调整后的测试图像形状:", resized_test_images.shape)


调整后的训练图像形状: (6000, 227, 227, 3)
调整后的测试图像形状: (1000, 227, 227, 3)


### 2 构建AlexNet模型

In [4]:

# 构建AlexNet模型
model = models.Sequential()
model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(227, 227, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Conv2D(256, (5, 5), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


### 3 训练AlexNet模型

In [5]:
# 编译和训练模型
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(resized_train_images, train_labels, epochs=10, batch_size=20, validation_data=(resized_test_images, test_labels))
# 电脑性能有限，仅选择部分数据进行模型的训练和测试

Epoch 1/10
300/300 [==============================] - 37s 123ms/step - loss: 2.8267 - accuracy: 0.6973 - val_loss: 1.6012 - val_accuracy: 0.6950
Epoch 2/10
300/300 [==============================] - 36s 120ms/step - loss: 0.9157 - accuracy: 0.8218 - val_loss: 1.3226 - val_accuracy: 0.6670
Epoch 3/10
300/300 [==============================] - 36s 121ms/step - loss: 0.6454 - accuracy: 0.8587 - val_loss: 0.6813 - val_accuracy: 0.8850
Epoch 4/10
300/300 [==============================] - 36s 121ms/step - loss: 0.7304 - accuracy: 0.8373 - val_loss: 0.6860 - val_accuracy: 0.9040
Epoch 5/10
300/300 [==============================] - 36s 121ms/step - loss: 0.5616 - accuracy: 0.8757 - val_loss: 0.4872 - val_accuracy: 0.9040
Epoch 6/10
300/300 [==============================] - 36s 121ms/step - loss: 0.6376 - accuracy: 0.8620 - val_loss: 1.0980 - val_accuracy: 0.8660
Epoch 7/10
300/300 [==============================] - 36s 121ms/step - loss: 0.6529 - accuracy: 0.8650 - val_loss: 0.4006 - val_ac

### 4 显示模型信息并保存为.h5文件

In [6]:
model.summary()
# 保存模型
model.save('alexnet.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8